
 #  <span style="color:#D47FAC;"> <center>LIST 3 </center> </span>

Authors: _Jakubik Małgorzata, Szymkowiak Magdalena_

In [66]:
import time

## <span style="color:#D47FAC;">TASK 1</span> 

Our task is to implement:
$$P(n,k)=\sum_{i=0}^{k}{n\choose i}p^i(1-p)^{n-i}$$

in a such way that execute only **3k+logn** multiplications.

We start with modification of the equation above, to get constant expression before the $\sum$:

$$\sum_{i=0}^{k}{n\choose i}p^i(1-p)^{n-i}=(1-p)^n\sum_{i=1}^k{n\choose i}\left( \frac{1}{1-p}\right)^i$$

Then we also modificate the form of the Binomial coefficient ${n\choose i}$, to cut off repetitive multiplications. 
$${n\choose k}=\frac{n^k}{k!}=\frac{n(n-1)(n-2)\dots(n-(k-1))}{k(k-1)(k-2)\dots1}=\prod_{i=1}^{k}\frac{n+1-i}{i}$$

Thanks to these transitions we can implement the probability without unnecessary multiplications.



Moreover, to decrease amout of multiplications we start loop from $i=2$. It is because:
- for $i=0$
$${n\choose 0}\left( \frac{1}{1-p}\right)^0=1$$

- for $i=1$
$${n\choose 1}\left( \frac{1}{1-p}\right)^1=n\left( \frac{1}{1-p}\right)$$

That is why at the begining we have  *count_mult=2*.

The rest is iterated in the loop. 

###### Exponentiation
To better compute the large positive integer powers of numbers we use exponentiation by squaring method. Raising $a$ to the power of $n$ allows to avoid multiplying a by itself $n-1$ times.
Computing powers using this algorithm has a $log(n)$ complexity.

We are looking for the powers $a^1, a^2, a^4, a^8, ..., a^{\lfloor{logn}\rfloor}$ in places we have corresponding bit in $n$.

Using this method, if the power is even we divide power by 2 and multiply base to itself. 

If the power is odd we decrement power by 1 to make it even and again multiply power by 2.

In [7]:
def power(base, power):
    """
    Calculate the power using exponentiation by squaring
    :param base: number which exponentiation is calculated
    :param power: exponent of the function
    :return: Number raised to the power
    """

    if power < 0:
        raise ValueError("Invalid value")
        
    if not isinstance(power, int):
        raise TypeError("Power should be an integer.")
        
    if power == 0:
        return 1
    
    result = 1
    while power > 0:
        # If power is odd
        if power % 2 == 1:
            result = (result * base)

        # Divide the power by 2
        power = power // 2
        # Multiply base to itself
        base = (base * base)

    return result

In [8]:
def probability(n, k, p):
    """
    Calculate the Binomial Distribution of k successed in n trials with p probability of success.
    
    :param n: amount of trials
    :param k: amount of successes
    :param p: probability of success
    :return: tuple with wanted probability and number of executed multiplications
    """
    
    if p < 0 or p >1:
        raise ValueError("Probability has a value between 0 and 1")
    
    x=p/(1-p) 
    
    prob=1
    binom_all_x=n*x #for i=1
    count_mult=2 #one multiplication done in x and one in binom_all_x
    
    for i in range(2,k+1):
        binom_all_x*=(n+1-i)/i*x #3 mult
        prob+=binom_all_x
        count_mult+=3
    
    factor = power(1-p,n)
    prob *= factor
    count_mult+=1 #1 mult done in return
    
    return (prob, count_mult)

In [68]:
print(probability(10, 7, 0.8))

(0.32219637759999953, 21)


As we can see, function *probability* made $21$ multiplications according to the **3k**.Because $3\cdot 7=21$.

## <span style="color:#D47FAC;">TASK 2</span> 

Program calculate value of polynomial, using the Horner's algorithm. It is more eficient than implementing straight as solving it on the paper.

At first we define *n* which is a number of figures in the list of *coeff*. Then we reverse our list *coeff*, to have coefficents positioned from the term of the highest degree to the free term. 

To *value* variable we assign the coefficient standing by the highest degree of polynomial.

The variable *count_mult* as the name indicates, counts multiplications out.

Next we applay the loop based on **Horner's rule**: 
$$a_0+a_1x+a_2x^2+a_3x^3+\dots+a_nx^n
=(((\dots(a_nx+a_{n-1}) x+\dots)x+a_1)x+a_0)$$.

In the loop we are increasing our variable *count_mult* by 1, because program does one multiplication with one iteration. The same situation is if it is going about adding, which *count_add* refers to it.

At the end our function returns a tuple with calculated value of the polynomial and amounts of executed adding and multiplications.

In [29]:
def smart_polynomial_value_calc(coeff, arg):
    """
    Calculate value of polynomial using the Horner's algorithm
    :param coeff: a list with coefficients of polynomial positioned from free term to the term of the highest degree
    :param arg: the value of variable x in polynomial W(x)
    :return: tuple with the value of polynomial and amout of executed multiplication and addition
    """
    if not isinstance(coeff,list):
        raise TypeError("Variable 'coeff' should be a list")
        
    if type(arg) not in {int,float}:
        raise TypeError("Variable 'arg' should be an int or float")
    
    if coeff==[]:
        raise ValueError("Lack of coefficients, fill list with numvers!")
        
    n=len(coeff)
    coeff.reverse() 
    value=coeff[0]
    count_mult=0
    for dg in range(1,n): #dg=degree
        value = value*arg + coeff[dg]
        count_mult+=1
    count_add=count_mult
    return value, count_mult, count_add

This program of solving polynomial is implemented straight as we solve polynomials on paper. 

In [30]:
def ordinary_polynomial_value_calc(coeff, arg):
    """
    Calculate value of polynomial in a standart way
    :param coeff: a list with coefficients of polynomial positioned from free term to the term of the highest degree
    :param arg: the value of variable x in polynomial W(x)
    :return: tuple with the value of polynomial and amout of executed multiplication and addition
    """
    if not isinstance(coeff,list):
        raise TypeError("Variable 'coeff' should be a list")
        
    if type(arg) not in {int,float}:
        raise TypeError("Variable 'arg' should be an int or float")
    
    if coeff==[]:
        raise ValueError("Lack of coefficients, fill list with numvers!")
    
    n=len(coeff)
    count_add=0
    count_mult=0
    value=coeff[0]
    for dg in range(1,n):
        value=coeff[dg]*(arg**dg) + value
        count_mult+=dg
        count_add+=1
    return value, count_mult, count_add
  

In [27]:
print(smart_polynomial_value_calc([1,2,3,4], 2))
print(ordinary_polynomial_value_calc([1,2,3,4], 2))

(49, 3, 3)
(49, 6, 3)


In [28]:
print(smart_polynomial_value_calc([20,5,-10,18], -1.5))
print(ordinary_polynomial_value_calc([20,5,-10,18], -1.5))

(-70.75, 3, 3)
(-70.75, 6, 3)


To sum up these two algorithms, the one where we used *Horner's rule* is more efficient. Executes less multiplications than in ordinary way. In small polynomial there is a small difference, however when we want to calculate huge polynomial, then the diffrence is significative.

## <span style="color:#D47FAC;">TASK 3</span> 

In this task we would like to count the numbers of charachters in a given text without using if-statement.

To do this we need to prepare text file:
<ul>
    <li>Delete all of whitespace charakters</li>
    <li>Make every letter lowercase</li>
</ul>

To avoid using if-statement we use for-loop to count apperance of a charackter and make dictionary.

In [91]:
def counting_chars_without_ifs(filename):
    """
    Count the number of all characters appearing in given file.
    :param fiename: file .txt with the text
    :return: dictionary with every character that appeared in the given file
    """
    file_ref = open(filename, 'r')
    text = file_ref.read()
    low_text = text.lower()
    striped_text = ''.join(low_text.split())
    char_count = {i:striped_text.count(i) for i in set(striped_text)}  
    return char_count

In [92]:
counting_chars_without_ifs('L3_ZAD3_sample_text.txt')

{',': 10,
 'r': 48,
 'l': 34,
 'j': 1,
 's': 51,
 ';': 4,
 'w': 21,
 'i': 62,
 't': 74,
 'v': 12,
 'd': 39,
 "'": 1,
 'u': 25,
 'q': 1,
 'p': 12,
 'y': 23,
 'a': 74,
 'f': 26,
 'e': 115,
 'm': 21,
 'g': 16,
 'c': 16,
 'h': 83,
 'k': 9,
 '-': 2,
 'n': 79,
 'b': 14,
 'o': 72,
 '.': 7}

#### <span style="color:#D47FAC;"> <center> Link to our code: https://github.com/maggszy/date_algorithm/blob/main/list3/lista3.ipynb </center> </span>